In [ ]:
# Install and import libraries
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

# Import dataset
!wget -q https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()


In [ ]:
# Convert categorical columns to numbers
dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'], drop_first=True)

# Normalize numeric columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numeric_cols = ['age', 'bmi', 'children']
dataset[numeric_cols] = scaler.fit_transform(dataset[numeric_cols])


In [ ]:
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')


In [ ]:
def build_model():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=[train_dataset.shape[1]]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])
    return model

model = build_model()
model.summary()


In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=20)

history = model.fit(
    train_dataset, train_labels,
    validation_split=0.2,
    epochs=500,
    callbacks=[early_stop],
    verbose=0
)

# Optional: visualize training progress
plt.plot(history.history['mae'], label='MAE (training data)')
plt.plot(history.history['val_mae'], label='MAE (validation data)')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.show()


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions
test_predictions = model.predict(test_dataset).flatten()
a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
